In [1]:
import pandas as pd
import geopandas as gpd
import folium, os

In [2]:
def set_color_tuple(row):
    total = sum(row.iloc[:][1:4])
    color_tup = (round(255*row.iloc[:][1]/total, 1), 
                 round(255*row.iloc[:][3]/total, 1),
                 round(255*row.iloc[:][2]/total, 1))
    return color_tup

def rgb_to_hex(rgb_tup):
    rgb_tup = tuple(round(x) for x in rgb_tup)
    return '#%02x%02x%02x' % rgb_tup


def get_data_by_state_year(state='MN', year=2012):
    filename = f'{state}{year}.txt'
    
    dat = pd.read_csv(f'../data/{filename}').drop('Unnamed: 4', axis=1)
    dat['rgb_color'] = dat.apply(lambda row: set_color_tuple(row), axis=1)
    dat['hex_color'] = dat.apply(lambda row: rgb_to_hex(row.rgb_color), axis=1)
    dat['Year'] = year
    dat['State'] = state
    dat = dat.rename(columns={f'{year} US Presidential Election':'County',
                             dat.columns[1]:'Republican',
                             dat.columns[2]:'Democrat',
                             dat.columns[3]:'Other'})
    
    cols = dat.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    
    return dat[cols]

def identify_results_files():
    
    listdir = os.listdir('../data')
    
    filenames = [filename for filename in listdir if len(filename)==10]
    
    return filenames

In [3]:
get_data_by_state_year()

,Year,State,County,Republican,Democrat,Other,rgb_color,hex_color
0,2012,MN,Aitkin,4533,4412,197,"(126.4, 5.5, 123.1)",#7e067b
1,2012,MN,Anoka,93430,88614,4421,"(127.8, 6.0, 121.2)",#800679
2,2012,MN,Becker,9204,6829,349,"(143.3, 5.4, 106.3)",#8f056a
3,2012,MN,Beltrami,9637,11818,596,"(111.4, 6.9, 136.7)",#6f0789
4,2012,MN,Benton,10849,8173,597,"(141.0, 7.8, 106.2)",#8d086a
...,...,...,...,...,...,...,...,...
82,2012,MN,Watonwan,2517,2494,133,"(124.8, 6.6, 123.6)",#7d077c
83,2012,MN,Wilkin,1884,1258,80,"(149.1, 6.3, 99.6)",#950664
84,2012,MN,Winona,11480,14980,772,"(107.5, 7.2, 140.3)",#6c078c
85,2012,MN,Wright,40466,25741,1609,"(152.2, 6.1, 96.8)",#980661


In [4]:
# Create initial dictionary so that I can create initial DataFrame
# so that I can concatenate to that DataFrame
setup_dict = dict.fromkeys(get_data_by_state_year().columns)
setup_dict['Year'] = []

results_df = pd.DataFrame.from_dict(setup_dict)

# results_df['Year'] = pd.to_numeric(results_df['Year'])

# results_df

results_filenames = identify_results_files()

counter = 1
print('File number: ', counter)

for filename in results_filenames:
    if counter % 25 == 0:
        print('File number: ', counter)
    counter += 1
    
    state_abbr = filename[:2]
    year = filename[2:6]
#     print(state_abbr, year)
    
    df = get_data_by_state_year(state_abbr, year)
#     print(df.head(1))
    
    results_df = results_df.append(df, ignore_index=True)
    results_df.reset_index(drop=True,inplace=True)
#     print(results_df.tail(5))
    

print('File number: ', counter-1)
results_df['Year'] = pd.to_numeric(results_df['Year'])
results_df = results_df.sort_values(by=['Year', 'State', 'County'])
results_df.reset_index(drop=True,inplace=True)

File number:  1
File number:  25
File number:  50
File number:  75
File number:  100
File number:  125
File number:  150
File number:  175
File number:  200
File number:  225
File number:  250
File number:  275
File number:  300
File number:  325
File number:  350
File number:  375
File number:  400
File number:  425
File number:  450
File number:  475
File number:  500
File number:  525
File number:  550
File number:  575
File number:  600
File number:  625
File number:  650
File number:  675
File number:  700
File number:  713


In [5]:
len(results_filenames)

713

In [6]:
results_df[results_df['Year'] == 2012]

,Year,State,County,Republican,Democrat,Other,rgb_color,hex_color
40805,2012,AK,District 1,5899,1518,305,"(194.8, 10.1, 50.1)",#c30a32
40806,2012,AK,District 10,4928,1344,298,"(191.3, 11.6, 52.2)",#bf0c34
40807,2012,AK,District 11,6057,2222,352,"(179.0, 10.4, 65.6)",#b30a42
40808,2012,AK,District 12,4300,1901,281,"(169.2, 11.1, 74.8)",#a90b4b
40809,2012,AK,District 13,3078,2425,204,"(137.5, 9.1, 108.4)",#8a096c
...,...,...,...,...,...,...,...,...
43956,2012,WY,Sweetwater,11428,4774,693,"(172.5, 10.5, 72.1)",#ac0a48
43957,2012,WY,Teton,4858,6213,393,"(108.1, 8.7, 138.2)",#6c098a
43958,2012,WY,Uinta,6615,1628,296,"(197.5, 8.8, 48.6)",#c60931
43959,2012,WY,Washakie,3014,794,136,"(194.9, 8.8, 51.3)",#c30933


In [7]:
def purple_states_of_america(election_results, county_json, year=2012):
    """INPUT: election_results: Dataframe of presidential election results, reported at the county level,
        year: year of presidential election
    OUTPUT: Map of county-level results, colored using RGB where Red level is based on percent of votes for
        the Republican candidate, Green level is based on the percent of votes for other candidate(s),
        and Blue level is based on percent of votes for the Democratic candidate.
    """
    
    election_results[election_results['Year'] == year]
    
    return election_results
    
#     m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

#     folium.GeoJson(
#         county_json,
#         style_function=lambda feature: {
#             'fillColor': ,
#             'color': 'black',
#             'weight': 2,
#             'dashArray': '5, 5'
#         }
#     ).add_to(m)

#     m

In [8]:
import geopandas as gpd

county_shapes_gdf = gpd.read_file('../data/US_counties_finer_boundaries.txt')

county_shapes_gdf.drop('id', axis=1, inplace=True)

county_shapes_gdf = county_shapes_gdf.rename(columns={'name': 'County',
                                       'state': 'State'})

county_shapes_gdf

,County,State,geometry
0,Autauga,AL,"POLYGON ((-86.91697 32.66403, -86.81659 32.659..."
1,Baldwin,AL,"POLYGON ((-87.76516 31.29718, -87.76043 31.297..."
2,Barbour,AL,"POLYGON ((-85.41029 32.14650, -85.25781 32.148..."
3,Bibb,AL,"POLYGON ((-87.06574 33.24679, -87.02685 33.246..."
4,Blount,AL,"POLYGON ((-86.45289 34.25922, -86.44415 34.259..."
...,...,...,...
3203,Sweetwater,WY,"POLYGON ((-110.05300 42.27080, -109.49607 42.2..."
3204,Teton,WY,"POLYGON ((-111.04820 44.47414, -111.05456 44.6..."
3205,Uinta,WY,"POLYGON ((-111.04510 41.57990, -110.04730 41.5..."
3206,Washakie,WY,"POLYGON ((-108.54989 44.16850, -107.74149 44.1..."


In [9]:
gdf = results_df.merge(county_shapes_gdf, on=['State','County'], how='left')
gdf

,Year,State,County,Republican,Democrat,Other,rgb_color,hex_color,geometry
0,1960,AK,District 1,180,257,0,"(105.0, 0.0, 150.0)",#690096,None
1,1960,AK,District 10,11119,9581,0,"(137.0, 0.0, 118.0)",#890076,None
2,1960,AK,District 11,568,634,0,"(120.5, 0.0, 134.5)",#780086,None
3,1960,AK,District 12,1005,1120,0,"(120.6, 0.0, 134.4)",#790086,None
4,1960,AK,District 13,631,867,0,"(107.4, 0.0, 147.6)",#6b0094,None
...,...,...,...,...,...,...,...,...,...
45328,2012,WY,Sweetwater,11428,4774,693,"(172.5, 10.5, 72.1)",#ac0a48,"POLYGON ((-110.05300 42.27080, -109.49607 42.2..."
45329,2012,WY,Teton,4858,6213,393,"(108.1, 8.7, 138.2)",#6c098a,"POLYGON ((-111.04820 44.47414, -111.05456 44.6..."
45330,2012,WY,Uinta,6615,1628,296,"(197.5, 8.8, 48.6)",#c60931,"POLYGON ((-111.04510 41.57990, -110.04730 41.5..."
45331,2012,WY,Washakie,3014,794,136,"(194.9, 8.8, 51.3)",#c30933,"POLYGON ((-108.54989 44.16850, -107.74149 44.1..."


In [10]:
gdf.dtypes

Year             int64
State           object
County          object
Republican      object
Democrat        object
Other           object
rgb_color       object
hex_color       object
geometry      geometry
dtype: object

In [11]:
gdf[gdf.geometry == None].State.unique()

array(['AK', 'HI', 'NV', 'SD', 'VA', 'CO', 'ME', 'RI'], dtype=object)

In [12]:
# No Alaska in USA-county.txt in files from Punch & Enbody course
# gdf[(gdf.geometry == None) & (gdf.State == 'AK')]

# No Hawaii in USA-county.txt in files from Punch & Enbody course
# gdf[(gdf.geometry == None) & (gdf.State == 'HI')]

# Ormbsy County, NV consolidated with Carson City, NV in 1969
# gdf[(gdf.geometry == None) & (gdf.State == 'NV')]

# Washabaugh County, SD merged with Jackson County, SD in 1983
# gdf[(gdf.geometry == None) & (gdf.State == 'SD')]

# Nansemond County merged with Suffolk in 1974
# No polygon for Clifton Forge City in the dataset from Punch & Enbody
# No polygon for South Boston City in the dataset from Punch & Enbody
# gdf[(gdf.geometry == None) & (gdf.State == 'VA')]

# Broomfield CO is both a county and city but
# no polygon for Clifton Forge City in the dataset from Punch & Enbody
# gdf[(gdf.geometry == None) & (gdf.State == 'CO')]

# gdf[(gdf.geometry == None) & (gdf.State == 'ME')]

# gdf[(gdf.geometry == None) & (gdf.State == 'RI')]

In [86]:
results_df_2012 = results_df.loc[results_df.Year == 2012].reset_index(drop=True)
# results_df_2012
results_2012 = county_shapes_gdf.merge(results_df_2012, on=['State','County'], how='left')
results_2012_json = county_shapes_gdf.merge(results_df_2012, on=['State','County'], how='left').to_json()

In [93]:
results_2012

,County,State,geometry,Year,Republican,Democrat,Other,rgb_color,hex_color
0,Autauga,AL,"POLYGON ((-86.91697 32.66403, -86.81659 32.659...",2012.0,17379,6363,231,"(184.9, 2.5, 67.7)",#b90244
1,Baldwin,AL,"POLYGON ((-87.76516 31.29718, -87.76043 31.297...",2012.0,66016,18424,1051,"(196.9, 3.1, 55.0)",#c50337
2,Barbour,AL,"POLYGON ((-85.41029 32.14650, -85.25781 32.148...",2012.0,5550,5912,55,"(122.9, 1.2, 130.9)",#7b0183
3,Bibb,AL,"POLYGON ((-87.06574 33.24679, -87.02685 33.246...",2012.0,6132,2202,86,"(185.7, 2.6, 66.7)",#ba0343
4,Blount,AL,"POLYGON ((-86.45289 34.25922, -86.44415 34.259...",2012.0,20757,2970,333,"(220.0, 3.5, 31.5)",#dc0420
...,...,...,...,...,...,...,...,...,...
3204,Sweetwater,WY,"POLYGON ((-110.05300 42.27080, -109.49607 42.2...",2012.0,11428,4774,693,"(172.5, 10.5, 72.1)",#ac0a48
3205,Teton,WY,"POLYGON ((-111.04820 44.47414, -111.05456 44.6...",2012.0,4858,6213,393,"(108.1, 8.7, 138.2)",#6c098a
3206,Uinta,WY,"POLYGON ((-111.04510 41.57990, -110.04730 41.5...",2012.0,6615,1628,296,"(197.5, 8.8, 48.6)",#c60931
3207,Washakie,WY,"POLYGON ((-108.54989 44.16850, -107.74149 44.1...",2012.0,3014,794,136,"(194.9, 8.8, 51.3)",#c30933


In [ ]:
mymap = folium.Map([38, -100], tiles='cartodbpositron', zoom_start=5)

districts = folium.GeoJson(
    results_2012,
    style_function = lambda feature: {
        'fillColor': feature['properties']['hex_color'],
        'fillOpacity':1 ,
        'color': 'grey',
        'weight': 1,
        'dashArray': '2, 5',
    }
)

def get_tooltip_alias(feature):
    aliases = [feature['properties']['Democrat'],
               feature['properties']['Republican'],
               feature['properties']['Other']]
    
    return aliases

districts.add_child(
    folium.features.GeoJsonTooltip(fields=['Democrat', 'Republican', 'Other'],
                                  aliases=)
)

districts.add_to(mymap)

mymap.render()

mymap

In [210]:
def get_tooltip_alias(feature):
    aliases = [feature['properties']['Democrat'],
               feature['properties']['Republican'],
               feature['properties']['Other']]
    
    return aliases

In [215]:
results_2012.dem_name

AttributeError: 'GeoDataFrame' object has no attribute 'dem_name'

In [209]:
mymap.save??

Signature: mymap.save(outfile, close_file=True, **kwargs)
Source:   
    def save(self, outfile, close_file=True, **kwargs):
        """Saves an Element into a file.

        Parameters
        ----------
        outfile : str or file object
            The file (or filename) where you want to output the html.
        close_file : bool, default True
            Whether the file has to be closed after write.
        """
        if isinstance(outfile, str) or isinstance(outfile, bytes):
            fid = open(outfile, 'wb')
        else:
            fid = outfile

        root = self.get_root()
        html = root.render(**kwargs)
        fid.write(html.encode('utf8'))
        if close_file:
            fid.close()
File:      /opt/anaconda3/lib/python3.8/site-packages/branca/element.py
Type:      method


In [138]:
def plot_purple_states(gdf):
    
    m = folium.Map([38, -100], tiles='cartodbpositron', zoom_start=5)

    folium.GeoJson(
        gdf,
        style_function=lambda feature: {
#             'fillColor': feature['properties']['hex_color'],
            'fillColor':'#000000',
            'color': 'grey',
            'weight': 1,
            'dashArray': '1, 5'
        }
    ).add_to(m)
    
    return m

In [ ]:
obama_romney_map = plot_purple_states(results_2012)
obama_romney_map

In [ ]:
geo_json_data['features'][0]['properties']['name']

In [ ]:
dat.head()

In [ ]:
geo_json_data['features'][0]['fillColor'] = 'black'

In [ ]:
geo_json_data.keys()

In [ ]:
for state_num in range(len(geo_json_data['features'])):
    state_name = geo_json_data['features'][state_num]['properties']['name']
    geo_json_data['features'][state_num]['fillColor'] = (dat.loc[dat['2012 US Presidential Election']==state_name].reset_index(drop=True)).iloc[0].hex_color

In [ ]:
def filter_state(state_name='Minnesota'):
    for num in range(len(geo_json_data['features'])):
        if state_name == geo_json_data['features'][num]['properties']['name']:
            return geo_json_data['features'][num]

In [ ]:
filter_state('Minnesota')

In [ ]:
district_json_data = {'type': 'FeatureCollection'}
filtered_value = filter_state('Minnesota')
district_json_data.update({'features':[filtered_value]})

district_json_data

In [ ]:
m = folium.Map([46, -93], tiles='cartodbpositron', zoom_start=7)

def add_district_to_map(district_data):
    folium.GeoJson(
        district_data,
        style_function=lambda feature: {
            'fillColor' : district_data['features'][0]['fillColor'],
            'weight': 0
        }
    ).add_to(m)

In [ ]:
add_district_to_map(district_json_data)

m

In [ ]:
url = 'https://github.com/python-visualization/folium/raw/master/tests'
us_counties = f'{url}/us-counties.json'

county_geo_json_data = json.loads(requests.get(us_counties).text)

In [ ]:
def filter_state_counties(state_name='Minnesota'):
    for num in range(len(county_geo_json_data['features'])):
        if state_name == county_geo_json_data['features'][num]['properties']['name']:
            return county_geo_json_data['features'][num]

In [ ]:
county_geo_json_data

In [ ]:
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=5)

folium.GeoJson(
    county_shapes_json,
    style_function=lambda feature: {
        'fillColor': 'green',
        'color': 'grey',
        'weight': 1,
        'dashArray': '1, 5'
    }
).add_to(m)

m

In [142]:
import geopandas as gpd

In [153]:
pd_county_shapes_json = gpd.read_file('../data/US_counties_finer_boundaries.txt')

pd_county_shapes_json.drop('id', axis=1, inplace=True)

pd_county_shapes_json

,name,state,geometry
0,Autauga,AL,"POLYGON ((-86.91697 32.66403, -86.81659 32.659..."
1,Baldwin,AL,"POLYGON ((-87.76516 31.29718, -87.76043 31.297..."
2,Barbour,AL,"POLYGON ((-85.41029 32.14650, -85.25781 32.148..."
3,Bibb,AL,"POLYGON ((-87.06574 33.24679, -87.02685 33.246..."
4,Blount,AL,"POLYGON ((-86.45289 34.25922, -86.44415 34.259..."
...,...,...,...
3201,Sweetwater,WY,"POLYGON ((-110.05300 42.27080, -109.49607 42.2..."
3202,Teton,WY,"POLYGON ((-111.04820 44.47414, -111.05456 44.6..."
3203,Uinta,WY,"POLYGON ((-111.04510 41.57990, -110.04730 41.5..."
3204,Washakie,WY,"POLYGON ((-108.54989 44.16850, -107.74149 44.1..."


In [ ]:
m = folium.Map([38, -95], tiles='cartodbpositron', zoom_start=5)

folium.Choropleth(
    results_2012,
    fill_color = '#FF0000',
    fill_opacity = 1,
).add_to(m)

m

In [179]:
m = folium.Map([38, -95], tiles='cartodbpositron', zoom_start=5)

folium.Choropleth(
    results_2012,
    fill_color = results_2012['hex_color'],
    fill_opacity = 1,
).add_to(m)

m

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
m2 = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=5)

folium.GeoJson(results_2012,
               style_function=lambda feature:{
                   'fillColor': '#ff0000',
                   'fillOpacity': 1,
                   'color': 'black',
                   'weight': 1,
                   'dashArray': '1, 5',
                  
              }).add_to(m2)

m2

In [ ]:
m3 = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=5)

folium.GeoJson(results_2012,
               style_function=lambda feature:{
                   'fillColor': feature['properties']['hex_color'],
                   'fillOpacity': 1,
                   'color': 'black',
                   'weight': 1,
                   'dashArray': '1, 5',
                  
              }).add_to(m3)

m3

In [156]:
folium.Choropleth??

Init signature:
folium.Choropleth(
    geo_data,
    data=None,
    columns=None,
    key_on=None,
    bins=6,
    fill_color=None,
    nan_fill_color='black',
    fill_opacity=0.6,
    nan_fill_opacity=None,
    line_color='black',
    line_weight=1,
    line_opacity=1,
    name=None,
    legend_name='',
    overlay=True,
    control=True,
    show=True,
    topojson=None,
    smooth_factor=None,
    highlight=None,
    **kwargs,
)
Source:        
class Choropleth(FeatureGroup):
    """Apply a GeoJSON overlay to the map.

    Plot a GeoJSON overlay on the base map. There is no requirement
    to bind data (passing just a GeoJSON plots a single-color overlay),
    but there is a data binding option to map your columnar data to
    different feature objects with a color scale.

    If data is passed as a Pandas DataFrame, the "columns" and "key-on"
    keywords must be included, the first to indicate which DataFrame
    columns to use, the second to indicate the layer in the GeoJSON
   

In [157]:
folium.Choropleth(results_2012)